<a href="https://colab.research.google.com/github/PaulinaRuizB/Machine_Learning_Theory/blob/main/Homework/Task_5_Probabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probabilidad marginal y condicional

En este ejercicio tenemos dos cajas (roja y azul), cada una con una cantidad de frutas (manzanas y naranjas).  
Además, tenemos la probabilidad de elegir cada caja:  
$$
P(B=r), \quad P(B=b)
$$

---

## 1. Probabilidades marginales de la fruta

La probabilidad de obtener una **manzana** al azar es:

$$
P(F=a) \;=\; P(B=r)\,P(F=a \mid B=r) \;+\; P(B=b)\,P(F=a \mid B=b)
$$

donde:

$$
P(F=a \mid B=r) \;=\; \frac{\text{# manzanas en caja roja}}{\text{total frutas en caja roja}}
$$

$$
P(F=a \mid B=b) \;=\; \frac{\text{# manzanas en caja azul}}{\text{total frutas en caja azul}}
$$

De manera análoga, para las **naranjas**:

$$
P(F=o) \;=\; P(B=r)\,P(F=o \mid B=r) \;+\; P(B=b)\,P(F=o \mid B=b)$$

---

## 2. Probabilidad condicional de la caja dado el tipo de fruta

Aplicamos el **teorema de Bayes**:

$$P(B=r \mid F=a) \;=\; \frac{P(F=a \mid B=r)\,P(B=r)}{P(F=a)}
$$

$$P(B=b \mid F=a) \;=\; \frac{P(F=a \mid B=b)\,P(B=b)}{P(F=a)}$$

$$P(B=r \mid F=o) \;=\; \frac{P(F=o \mid B=r)\,P(B=r)}{P(F=o)}$$

$$P(B=b \mid F=o) \;=\; \frac{P(F=o \mid B=b)\,P(B=b)}{P(F=o)}$$


In [1]:
# Funciones para proabilidades

def _validate(boxes, p_box): #validar los datos ingresados
    if abs(sum(p_box.values()) - 1.0) > 1e-12:
        raise ValueError("Las probabilidades de caja deben sumar 1.")
    for b, fruits in boxes.items():
        if any(v < 0 for v in fruits.values()):
            raise ValueError(f"Hay cantidades negativas en la caja '{b}'.")
        if sum(fruits.values()) == 0:
            raise ValueError(f"La caja '{b}' está vacía; no se puede muestrear.")

def _totales_por_caja(boxes):
    return {b: sum(frutas.values()) for b, frutas in boxes.items()}

def marginales_fruit(boxes, p_box):
    """
    boxes: dict de cajas -> dict de fruta -> cantidad (>=0)
           p.ej.: {"r": {"a":2, "o":6}, "b":{"a":3, "o":1}}
    p_box: dict de P(B=b) que suma 1, p.ej.: {"r":0.4, "b":0.6}
    return: dict fruit -> P(F=fruit)
    """
    _validate(boxes, p_box)
    total = _totales_por_caja(boxes)

    # recolectar todos los tipos de fruta que aparecen en alguna caja
    tipos = set().union(*[set(frutas.keys()) for frutas in boxes.values()])

    p_fruit = {}
    for f in tipos:
        p_f = 0.0
        for b in boxes:
            cnt_f = boxes[b].get(f, 0)
            p_f += p_box[b] * (cnt_f / total[b])
        p_fruit[f] = p_f
    return p_fruit

def cond_box_dado_fruit(boxes, p_box, fruit):
    """
    Devuelve P(B=b | F=fruit) para todas las cajas.
    """
    _validate(boxes, p_box)
    total = _totales_por_caja(boxes)
    p_fruit_all = marginales_fruit(boxes, p_box)

    if p_fruit_all.get(fruit, 0.0) == 0.0:
        raise ValueError(f"P(F={fruit}) = 0; la fruta no aparece en ninguna caja.")

    res = {}
    for b in boxes:
        p_f_dado_b = boxes[b].get(fruit, 0) / total[b]
        res[b] = p_f_dado_b * p_box[b] / p_fruit_all[fruit]  # Bayes
    return res

def resumen(boxes, p_box):
    """
    Devuelve:
      - P(B=b) (igual a p_box)
      - P(F=f) para cada fruta
      - P(B=b | F=f) para cada fruta
    """
    p_f = marginales_fruit(boxes, p_box)
    cond = {f: cond_box_dado_fruit(boxes, p_box, f) for f in p_f.keys()}
    return p_box, p_f, cond


# ======================
# EJEMPLO DEL EJERCICIO
# ======================
boxes = {
    "r": {"a": 2, "o": 6},  # caja roja
    "b": {"a": 3, "o": 1},  # caja azul
}
p_box = {"r": 0.4, "b": 0.6}

pB, pF, cond = resumen(boxes, p_box)

print("P(B) =", pB)
print("P(F) =", pF)
print("P(B | F):")
for f, d in cond.items():
    for b, v in d.items():
        print(f"  P(B={b} | F={f}) = {v:.6f}")


P(B) = {'r': 0.4, 'b': 0.6}
P(F) = {'o': 0.45000000000000007, 'a': 0.5499999999999999}
P(B | F):
  P(B=r | F=o) = 0.666667
  P(B=b | F=o) = 0.333333
  P(B=r | F=a) = 0.181818
  P(B=b | F=a) = 0.818182
